# Games-Howell test

_The Games-Howell post-hoc test is another nonparametric approach to compare combinations of groups or treatments. Although rather similar to Tukey’s test in its formulation, the Games-Howell test does not assume equal variances and sample sizes. The test was designed based on Welch’s degrees of freedom correction and uses Tukey’s studentized range distribution, denoted q. The Games-Howell test is performed on the ranked variables similar to other nonparametric tests. Since the Games-Howell test does not rely on equal variances and sample sizes, it is often recommended over other approaches such as Tukey’s test._

In [1]:
import numpy as np
import pandas as pd
import math
from statsmodels.stats.libqsturng import psturng, qsturng
from itertools import combinations

In [2]:
def make_dicts(names, *columns):
    """
    names: np.array, names of the groups
    columns: np.array, corresponding data    
    """
    
    data = dict(zip(names, columns))
    n = []
    Mean = []
    std = []
    for col in columns:
        n.append(len(col))
        Mean.append(col.mean())
        std.append((np.var(col, ddof=1))**2)
    nd = dict(zip(names, n))
    Md = dict(zip(names, Mean))
    stdd = dict(zip(names, std))
    return data,nd,Md,stdd

def mean_dif(dctry_m, grp):
    key1 = grp[0]
    key2 = grp[1]
    return dctry_m[key1] - dctry_m[key2]

def df(dctry_s,dctry_n, grp):
    key1=grp[0]
    key2=grp[1]
    num = (dctry_s[key1]/dctry_n[key1] + dctry_s[key2]/dctry_n[key2])**2
    den = ((dctry_s[key1]/dctry_n[key1])**2)/(dctry_n[key1]-1) + ((dctry_s[key2]/dctry_n[key2])**2)/(dctry_n[key2]-1)
    return num/den

def t(dctry_m, dctry_s, dctry_n, grp):
    key1 = grp[0]
    key2 = grp[1]
    num = abs(mean_dif(dctry_m, grp))
    den = math.sqrt(dctry_s[key1]/dctry_n[key1] + dctry_s[key2]/dctry_n[key2])
    return num/den

def sigma(dctry_s, dctry_n, grp):
    key1 = grp[0]
    key2 = grp[1]
    return math.sqrt(0.5**(dctry_s[key1]/dctry_n[key1] + dctry_s[key2]/dctry_n[key2]))

def p(dctry_m, dctry_s, dctry_n, grp):
    K = len(dctry_m)
    T = t(dctry_m, dctry_s, dctry_n, grp) * math.sqrt(2)
    DF = df(dctry_s,dctry_n, grp)
    return psturng(T, K, DF)

def upper(dctry_m, dctry_s, dctry_n, grp):
#     P = p(dctry_m, dctry_s, dctry_n, grp)
    P = 0.95
    K = len(dctry_m)
    DF = df(dctry_s,dctry_n, grp)
    return mean_dif(dctry_m, grp) + qsturng(P, K, DF)*sigma(dctry_s, dctry_n, grp)

def lower(dctry_m, dctry_s, dctry_n, grp):
#     P = p(dctry_m, dctry_s, dctry_n, grp)
    P = 0.95
    K = len(dctry_m)
    DF = df(dctry_s,dctry_n, grp)
    return mean_dif(dctry_m, grp) - qsturng(P, K, DF)*sigma(dctry_s, dctry_n, grp)


#data = dctry[0]
# nd = dctry[1]
# Md = dctry[2]
# stdd = dctry[3]

def Games_Howell(names, *columns):
    """
    names: np.array, names of the groups
    columns: np.array, corresponding data   
    """
    dctry = make_dicts(names, *columns)
    groups = [list(x) for x in combinations(dctry[0].keys(), 2)]
    N = len(groups)
    MEAN_DIF = np.zeros(N)
    SIGMA = np.zeros(N)
    T = np.zeros(N)
    DF = np.zeros(N)
    P = np.zeros(N)
    UPPER = np.zeros(N)
    LOWER = np.zeros(N)
    
    for i in range(N):
        grp = groups[i]
        MEAN_DIF[i] = mean_dif(dctry[2], grp)
        SIGMA[i] = sigma(dctry[3], dctry[1], grp)
        T[i] = t(dctry[2], dctry[3], dctry[1], grp)
        DF[i] = df(dctry[3], dctry[1], grp)
        P[i] = p(dctry[2], dctry[3], dctry[1], grp)
        UPPER[i] = upper(dctry[2], dctry[3], dctry[1], grp)
        LOWER[i] = lower(dctry[2], dctry[3], dctry[1], grp)
    
    
    data ={'group':groups,'Mean Difference':MEAN_DIF, 'Standard Error':SIGMA, 't':T, 'df':DF, 
           'p':P, 'Lower limit':LOWER, 'Upper limit':UPPER,}
    
    return pd.DataFrame(data=data)


# Applications

#### *Example 1*

In [3]:
names = np.array(['a','b','c'])
a = np.array([1,1,1])
b = np.array([1,2,2,1])
c = np.array([5,6,7,6,5])
d = make_dicts(names,a,b,c)

Games_Howell(names, a,b,c)

,group,Mean Difference,Standard Error,t,df,p,Lower limit,Upper limit
0,"[a, b]",-0.5,0.990419,3.000000,3.000000,0.112472,-6.347195,5.347195
1,"[a, c]",-4.8,0.966606,15.333038,4.000000,0.001000,-9.665080,0.065080
2,"[b, c]",-4.3,0.957345,12.124574,5.951411,0.001000,-8.459073,-0.140927


#### *Example 2*

In [4]:
names1 = np.array(['a1', 'a2', 'a3'])
a1 = np.random.normal(0,2,80)
a2 = np.random.normal(100,1,80)
a3 = np.random.normal(1,1,80)
w = {'a1':a1,'a2':a2,'a3':a3}
df1 = pd.DataFrame(w)
Games_Howell(names1, df1['a1'], df1['a2'], df1['a3'])

,group,Mean Difference,Standard Error,t,df,p,Lower limit,Upper limit
0,"[a1, a2]",-99.707387,0.910375,191.556101,85.191721,0.001000,-102.778551,-96.636223
1,"[a1, a3]",-0.591627,0.910583,1.138005,84.794873,0.494916,-3.663743,2.480490
2,"[a2, a3]",99.115760,0.993159,704.267358,157.826005,0.001000,95.792325,102.439196


#### *Example 3*

In [5]:
names2 = np.array(['r1','r2','r3', 'r4'])
r1 = np.random.normal(0,3,70)
r2 = np.random.normal(2, 1, 30)
r3 = np.random.normal(15, 5, 60)
r4 = np.random.normal(15, 1, 70)
Games_Howell(names2, r1,r2,r3,r4)

,group,Mean Difference,Standard Error,t,df,p,Lower limit,Upper limit
0,"[r1, r2]",-1.878403,0.524573,1.376731,71.590454,0.515989,-3.829859,0.073053
1,"[r1, r3]",-14.828503,0.009278,4.035240,77.286931,0.001000,-14.862960,-14.794047
2,"[r1, r4]",-14.851720,0.527355,10.930121,70.473807,0.001000,-16.814264,-12.889175
3,"[r2, r3]",-12.950101,0.017266,3.784092,59.350892,0.002001,-13.014649,-12.885553
4,"[r2, r4]",-12.973317,0.981361,55.679162,62.412315,0.001000,-16.636826,-9.309807
5,"[r3, r4]",-0.023216,0.017358,0.006788,59.197198,0.900000,-0.088111,0.041679
